## Simple Data Generation

Process:
- Build campaign list
- For each date within campaign period, generate:
    - impressions / clicks / conversions
    - user events

Need to be able to generate:
- CAC - cost to acquire each customer
- LTV expected lifetime value of that acquired customer
- Conversion rate by funnel
- Return on Ad Spend

### Core Schema:
- 2 Dim, 2 Fact
- Dims:
    - dim_campaigns
    - dim_date
- Facts:
    - fact_ad_performance
    - fact_customer_revenue

In [35]:
import pandas as pd
import numpy as np


#load data
campaigns = pd.read_csv('./dim_campaigns.csv')
dates = pd.read_csv('./dim_date.csv')

In [36]:
#generate fact_ad_performance

ad_perf_rows = []
for _, c in campaigns.iterrows():
    # c is a Series (row) from the campaigns dataframe
    campaign_days = pd.date_range(c['start_date'], c['end_date'])
    if len(campaign_days) == 0:
        continue
    daily_budget = c['budget_usd'] / len(campaign_days)
    for day in campaign_days:
        impressions = np.random.randint(10000, 50000)
        
        #clickthrough rate
        ctr = np.random.uniform(0.008, 0.03)
        clicks = int(impressions * ctr)
        
        #conversion rate
        cvr = np.random.uniform(0.01, 0.05)
        conversions = int(clicks * cvr)

        spend = daily_budget * np.random.uniform(0.9, 1.1)
        ad_perf_rows.append({
            'campaign_id': int(c['campaign_id']),
            'date': pd.to_datetime(day).date(),
            'impressions': int(impressions),
            'clicks': int(clicks),
            'conversions': int(conversions),
            'spend': float(spend)
        })

# convert to DataFrame for easier downstream processing
ad_perf_df = pd.DataFrame(ad_perf_rows)

ad_perf_df.to_csv('fact_ad_performance.csv', index=False)

In [37]:
#handle variable revenue + churn for those who are monthly vs. annual

# Generate fact_customer_revenue
cust_rows = []
cust_id = 1
for row in ad_perf_rows:
    campaign_id = row['campaign_id']
    date_ = row['date']
    conversions = row['conversions']
    for _ in range(conversions):
        plan = np.random.choice(["monthly", "annual"], p=[0.6, 0.4])
        months_active = np.random.randint(1, 12) if plan == "monthly" else 12
        churned = np.random.choice([True, False], p=[0.3, 0.7])
        revenue = round(14.99*months_active,0) if plan == "monthly" else 100
        cust_rows.append({
            'customer_id': cust_id,
            'campaign_id': campaign_id,
            'date_acquired': date_,
            'plan': plan,
            'revenue': revenue,
            'months_active': int(months_active),
            'churned': bool(churned)
        })
        cust_id += 1

cust_df = pd.DataFrame(cust_rows)

cust_df.to_csv('fact_customer_revenue.csv', index=False)